# `hiRes`

### Chris Hamm
### 2023-08030


A programs to increase the resolution of old digital photos

In [1]:
# Preliminaries

import torch
import cv2
from PIL import Image
import numpy as np
from RealESRGAN import RealESRGAN

/home/chris/Projects/transformerForecasting/transformer-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# This is from https://github.com/ai-forever/Real-ESRGAN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RealESRGAN(device, scale=4)
model.load_weights('/home/chris/Projects/Real-ESRGAN/weights/RealESRGAN_x4.pth', download=True)

#path_to_image = 'input/MaryBlairMonsanto.JPG'
image = Image.open(path_to_image).convert('RGB')

In [16]:
# enhance and save the  image
sr_image = model.predict(image)

#sr_image.save("output/enhanced_MaryBlairMonsanto.JPG")


# Now for movies!

In [2]:
# this one was mostly me, a little pieced to gether by seeing you branched the above repo

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RealESRGAN(device, scale=4)
model.load_weights('/home/chris/Projects/Real-ESRGAN/weights/RealESRGAN_x4.pth', download=True)
input_video_path = 'input/Monsanto_Night.mp4'
output_video_path = 'output/enhanced_Monsanto_Night.mp4'

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width * 4, frame_height * 4))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    sr_image = model.predict(image)
    sr_frame = np.array(sr_image)
    
    out.write(sr_frame)
    
cap.release()
out.release()

KeyboardInterrupt: 